In [1]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2023-06-29 09:00:36--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 142.251.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  9.17MB/s    in 0.6s    

2023-06-29 09:00:36 (9.17 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



In [27]:
import json
import numpy as np

In [28]:
with open('sarcasm.json','r') as f:
  datastore=json.load(f)

In [39]:
sentances=[]
labels=[]
for item in datastore:
  sentances.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [40]:
len(sentances)

26709

In [41]:
training_size=20000
training_sequences=sentances[:training_size]
testing_sequences=sentances[training_size:]
train_labels=labels[:training_size]
test_labels=labels[training_size:]

In [42]:
train_label=np.array(train_labels)
test_label=np.array(test_labels)

In [43]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
tokenizer=Tokenizer(num_words=1000,oov_token='<oov>')
tokenizer.fit_on_texts(training_sequences)
word_index=tokenizer.word_index
train_sequences=tokenizer.texts_to_sequences(training_sequences)
test_sequences=tokenizer.texts_to_sequences(testing_sequences)
train_pad=pad_sequences(train_sequences,maxlen=120,truncating='post',padding='post')
test_pad=pad_sequences(test_sequences,maxlen=120,truncating='post',padding='post')

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense,LSTM

In [46]:
model=Sequential()
model.add(Embedding(1000,16,input_length=120))
model.add(GRU(32,activation='relu'))
model.add(Dense(24,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [47]:
model.fit(train_pad,train_label,validation_data=(test_pad,test_label),epochs=10)

Epoch 1/10
625/625 [==============================] - 35s 52ms/step - loss: 0.6866 - accuracy: 0.5599 - val_loss: 0.6852 - val_accuracy: 0.5633
Epoch 2/10
625/625 [==============================] - 31s 49ms/step - loss: 0.6861 - accuracy: 0.5603 - val_loss: 0.6853 - val_accuracy: 0.5633
Epoch 3/10
625/625 [==============================] - 33s 52ms/step - loss: 0.6861 - accuracy: 0.5603 - val_loss: 0.6852 - val_accuracy: 0.5633
Epoch 4/10
625/625 [==============================] - 30s 48ms/step - loss: 0.6859 - accuracy: 0.5603 - val_loss: 0.6854 - val_accuracy: 0.5633
Epoch 5/10
625/625 [==============================] - 30s 48ms/step - loss: 0.6860 - accuracy: 0.5603 - val_loss: 0.6859 - val_accuracy: 0.5633
Epoch 6/10
625/625 [==============================] - 31s 50ms/step - loss: 0.6860 - accuracy: 0.5603 - val_loss: 0.6855 - val_accuracy: 0.5633
Epoch 7/10
625/625 [==============================] - 30s 48ms/step - loss: 0.6860 - accuracy: 0.5603 - val_loss: 0.6857 - val_accuracy:

In [51]:
pred=(model.predict(test_pad)>0.5).astype(int)

210/210 [==============================] - 3s 13ms/step


In [49]:
from sklearn.metrics import classification_report

In [58]:
print(classification_report(test_label,pred,zero_division=0))

              precision    recall  f1-score   support

           0       0.56      1.00      0.72      3779
           1       0.00      0.00      0.00      2930

    accuracy                           0.56      6709
   macro avg       0.28      0.50      0.36      6709
weighted avg       0.32      0.56      0.41      6709

